**EL ENUNCIADO DE ESTE EJERCICIO PIDE RESOLVERLO CON PANDAS**

**SE AGREGA LA RESOLUCIÓN DE SPARKSQL DE TODAS FORMAS**

> https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqu7n114zx

El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud).  

Los  posibles estados  que puede tener  un caso son 1: caso abierto, 2: caso  resuelto, 3: cerrado sin resolución.  

Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para 
pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respu
esta positiva (1) o negativa (0) de parte de él.

El  sector  encargado  de  las  estadísticas  oficiales  del  GCPD  quiere  con  esta  información  analizar  las  siguientes situaciones: 

- Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman). 
    
---

In [45]:
# Set-up y vista rápida de las dos bases de datos truchas
import pyspark

spark = pyspark.sql.SparkSession.builder.appName("Batman").getOrCreate()
df_gcpd = spark.read.csv('../data/2018C1_GCPD.csv', header=True)
df_gcpd.createOrReplaceTempView('GCPD')
df_gordon = spark.read.csv('../pandas/data/2018C1_gordon.csv', header=True)
df_gordon.createOrReplaceTempView('GORDON')

In [83]:
query = "SELECT GCPD.categoria, (SUM(case when GCPD.estado_caso=2 then 1 else 0 end) / COUNT(GCPD.estado_caso)) as tasa_resolucion \
        from GCPD left join GORDON on GCPD.id_caso = GORDON.id_caso \
        where GORDON.respuesta=0 or GORDON.respuesta is null \
        group by GCPD.categoria"

spark.sql(query).show()

+---------+------------------+
|categoria|   tasa_resolucion|
+---------+------------------+
|     Robo|0.3333333333333333|
|   Arseny|               0.0|
| Incendio|               0.0|
+---------+------------------+

